In [1]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.9 MB/s eta 0:00:00


In [5]:
import numpy as np

In [2]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import torch
from transformers import CLIPProcessor, CLIPModel
import faiss
import os

In [4]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

df = pd.read_csv("images.csv")

catalog_embeddings = []
catalog_metadata = []

for _,row in df.iterrows():  # start a loop through all rows
    try:
        response = requests.get(row["image_url"], timeout = 10)   # download the image using its URL
        image = Image.open(BytesIO(response.content)).convert("RGB")  # convert to RGB channels

        inputs = clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            emb = clip_model.get_image_features(**inputs)  # returns the vector for the image — a numerical summary of what the image contains.
        emb = emb / emb.norm(p=2, dim=-1, keepdim=True)  #  normalizes the vector, which is needed for cosine similarity (this step ensures all vectors have the same length).

        catalog_embeddings.append(emb.cpu().numpy()) # converts the PyTorch tensor to a NumPy array (needed for FAISS).
        catalog_metadata.append({"product_id": row["id"], "image_url": row["image_url"]})  # We store the embedding and its metadata (product ID, URL) in o

    except Exception as e:
        print(f"Failed to process {row['image_url']} - {e}")

Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1948_1600x.jpg?v=1738757403 - cannot identify image file <_io.BytesIO object at 0x7a71265b6700>
Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1968_1600x.jpg?v=1738757402 - cannot identify image file <_io.BytesIO object at 0x7a71262797b0>
Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1971_1600x.jpg?v=1738757403 - cannot identify image file <_io.BytesIO object at 0x7a71262784a0>
Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1973_1600x.jpg?v=1738757403 - cannot identify image file <_io.BytesIO object at 0x7a71265b5620>
Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1977_86dfa1dd-099e-4b31-8b42-0e3fda8d204e_1600x.jpg?v=1738757403 - cannot identify image file <_io.BytesIO object at 0x7a71265b62f0>
Failed to process https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1988E4_1600x.jpg?v=17

In [6]:
emb_matrix = np.concatenate(catalog_embeddings).astype("float32")  #  combines all embeddings into one big matrix.
index = faiss.IndexFlatIP(emb_matrix.shape[1])  # cosine similarity
index.add(emb_matrix)

faiss.write_index(index, "catalog.index")

# Save metadata
import json
with open("catalog_metadata.json", "w") as f:
    json.dump(catalog_metadata, f)

In [7]:
def crop_image(image,box):
    x1, y1, x2, y2 = map(int, box)
    return image.crop((x1, y1, x2, y2))

In [8]:
def get_clip_embedding(image_pil):
    inputs = clip_processor(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        emb = clip_model.get_image_features(**inputs)
    return emb / emb.norm(p=2, dim=-1, keepdim=True)

In [9]:
def match_to_catalog(embedding, index, catalog_metadata):
    emb_np = embedding.cpu().numpy().astype("float32")
    D, I = index.search(emb_np, k=1)  # top 1 match
    sim = float(D[0][0])
    match_info = catalog_metadata[I[0][0]]

    if sim > 0.9:
        label = "Exact Match"
    elif sim > 0.75:
        label = "Similar Match"
    else:
        label = "No Match"

    return {
        "product_id": match_info["product_id"],
        "image_url": match_info["image_url"],
        "similarity": sim,
        "label": label
    }

In [10]:
import json
import os
from PIL import Image

# Load full detections list
with open("detections.json") as f:
    all_detections = json.load(f)

# Convert bbox format for all detections
for det in all_detections:
    x, y, w, h = det["bbox"]
    det["bbox"] = [x, y, x + w, y + h]

# Get unique sorted frame numbers
frame_numbers = sorted(set(det["frame"] for det in all_detections))

# Prepare frame paths
frames_folder = "VIDEO FRAMES"
all_frame_paths = [os.path.join(frames_folder, f"frame_{frame:04d}.jpg") for frame in frame_numbers]

# Group detections by frame
frame_to_detections = {}
for det in all_detections:
    frame = det["frame"]
    frame_to_detections.setdefault(frame, []).append(det)

In [11]:
final_results = []

for frame_path in all_frame_paths:
    frame_num = int(os.path.basename(frame_path).split("_")[1].split(".")[0])
    image = Image.open(frame_path).convert("RGB")
    detections_in_frame = frame_to_detections.get(frame_num, [])

    for detection in detections_in_frame:
        crop = crop_image(image, detection['bbox'])
        emb = get_clip_embedding(crop)
        match = match_to_catalog(emb, index, catalog_metadata)

        final_results.append({
            "frame": detection["frame"],
            "bbox": detection["bbox"],
            "matched_product": match["product_id"],
            "similarity": match["similarity"],
            "label": match["label"],
            "product_image": match["image_url"]
        })

In [12]:
final_results

[{'frame': 0,
  'bbox': [301, 321, 449, 437],
  'matched_product': 15409,
  'similarity': 0.8605747818946838,
  'label': 'Similar Match',
  'product_image': 'https://cdn.shopify.com/s/files/1/0785/1674/8585/files/13th_NOV_2024_Virgio-1777_1600x.jpg?v=1734705540'},
 {'frame': 0,
  'bbox': [-3, 325, 679, 1245],
  'matched_product': 15209,
  'similarity': 0.8204257488250732,
  'label': 'Similar Match',
  'product_image': 'https://cdn.shopify.com/s/files/1/0785/1674/8585/files/4thDEC2024Virgio-0328_1600x.jpg?v=1733987468'},
 {'frame': 0,
  'bbox': [476, 377, 626, 895],
  'matched_product': 15409,
  'similarity': 0.811388373374939,
  'label': 'Similar Match',
  'product_image': 'https://cdn.shopify.com/s/files/1/0785/1674/8585/files/13th_NOV_2024_Virgio-1777_1600x.jpg?v=1734705540'},
 {'frame': 0,
  'bbox': [-10, 732, 679, 1275],
  'matched_product': 16720,
  'similarity': 0.8908952474594116,
  'label': 'Similar Match',
  'product_image': 'https://cdn.shopify.com/s/files/1/0785/1674/8585/fi